In [40]:
import advent

maps = advent.get_lines_doublenewline(5)


In [41]:
def is_in_range(range: list[int], num: int):
    # range looks like "x y z"
    _, source, length = range
    return num >= int(source) and num < (int(source) + int(length))

def apply_range(range: list[int], num: int):
    # ASSUMES ITS IN RANGE!
    dest, source, _ = range
    return num + int(dest) - int(source)

def apply_map(map: list[str], num: int):
    # assume map[0] is just a string, ignore it
    for range in map[1:]:
        range = [int(foo) for foo in range.split(" ")]
        if is_in_range(range, num): return apply_range(range, num)
    return num

print(apply_map(['seed-to-soil map:', '50 98 2', '52 50 48'], 99))
# should be 51

51


In [42]:
def apply_all_maps(data: list[list[str]], num: int):
    for map in data[1:]:
        num = apply_map(map, num)
    return num

nums = maps[0][0].split(": ")[1].split(" ")

results = [apply_all_maps(maps, int(num)) for num in nums]
print(min(results))

178159714


In [43]:
# part 2 idea:
# lets say we finished calculating 100, and the result was 42 (whatever)
# now 101 will logically be mapped to 43, meaning we don't need to even calculate it
# unless, 101 is in a new range, e.g. the first map was "20 99 2" (meaning 101 will be mapped somewhere else)
# to calculate when the next 'new range' begins, we calculate the headroom during each step
# e.g. the headroom during the first map was 20 and during the second map was 10, we can skip to 110 right away

def apply_range_headroom(range: list[int], num: int):
    # also returns headroom. still assumes the num must be in range
    dest, source, length = range
    result = num + dest - source
    headroom = source + length - num
    return result, headroom

# if the number is not in any range, the 'headroom' is the distance to the next source number
def outside_range_headroom(map: list[str], num: int):
    sources = [[int(foo) for foo in range.split(" ")][1] for range in map[1:]]
    if num > max(sources): return 2**1000 # basically this map doesnt contribute to headroom
    return min(source for source in sources if source > num) - num

def apply_map_headroom(map: list[str], num: int):
    # assume map[0] is just a string, ignore it
    for range in map[1:]:
        range = [int(foo) for foo in range.split(" ")]
        if is_in_range(range, num): return apply_range_headroom(range, num)
    return num, outside_range_headroom(map, num)

assert apply_range_headroom([20, 100, 10], 105) == (25, 5)
assert outside_range_headroom(['name', '10 20 30', '20 8 0'], 5) == 3

In [44]:
def apply_all_maps_headroom(data: list[list[str]], num: int):
    headroom: int = 2**1000 # just a random large number to start with
    for map in data[1:]:
        num, headroom_tmp = apply_map_headroom(map, num)
        if headroom_tmp < headroom: headroom = headroom_tmp # final headroom is the minimum of all headrooms
    return num, headroom

def apply_all_maps_headroom_to_range(data, start, length):
    num = start
    best_location = 2**1000
    while num < (start+length):
        location, headroom = apply_all_maps_headroom(data, num)
        if location < best_location: best_location = location
        num += headroom
    return best_location

best_location = 2**1000
for ix in range(0, len(nums), 2):
    start, length = int(nums[ix]), int(nums[ix+1])
    location = apply_all_maps_headroom_to_range(maps, start, length)
    if location < best_location: best_location = location

print(best_location)


100165128
